In [6]:
from mpgm.mpgm.evaluation.generating_samples import *
from mpgm.mpgm.models.SPGM import SPGM
from mpgm.mpgm.evaluation.evaluation_metrics import EvalMetrics
import matplotlib.pyplot as plt
import numpy as np
import time
from mpgm.mpgm.model_fitters.prox_grad_fitters import Prox_Grad_Fitter, Constrained_Prox_Grad_Fitter
from mpgm.mpgm.evaluation.evaluation_metrics import *

from mpgm.mpgm.sample_generation.samplers import *

from mpgm.mpgm.evaluation.preprocessing import ClampMax

from mpgm.mpgm.model_fitters.prox_grad_fitters import Prox_Grad_Fitter, Constrained_Prox_Grad_Fitter

In [12]:
nr_variables = 10
nr_samples = 150
SPW = SampleParamsWrapper(nr_variables=nr_variables,
                          sample_init=np.zeros((nr_variables,)),
                          nr_samples=nr_samples,
                          random_seed=1)

SPW.graph_generator = LatticeGraphGenerator(sparsity_level=0)
SPW.model = TPGM(R=10)
SPW.sampler = TPGMGibbsSampler(burn_in=200, thinning_nr=150)
SPW.weight_assigner = Constant_Weight_Assigner(ct_weight=0.1)
SPW.generate_samples_and_save("SPGM_test_samples", "samples.sqlite")

100%|██████████| 22551/22551 [00:40<00:00, 558.01it/s]

Generated samples: SPGM_test_samples


In [13]:
SPS = SampleParamsWrapper.load_samples("SPGM_test_samples", "samples.sqlite")
samples = SPS.samples

In [14]:
samples_file_name = "../samples.sqlite"
fit_file_name = "../fit_models.sqlite"
nr_variables = 10
nr_samples = 150
alpha = 0.5

samples_id = "SPGM_debug"
FPW = FitParamsWrapper(random_seed=0,
                       samples_file_name=samples_file_name)
FPW.model = SPGM(R=10, R0=5)

FPW.fitter = Prox_Grad_Fitter(alpha=0.1,
                              save_regularization_paths=True,
                              init_step_size=1,
                              early_stop_criterion='likelihood')

FPW.preprocessor = ClampMax(10)

theta_init = np.random.normal(0, 0.1, (nr_variables, nr_variables))
theta_fit = FPW.fit_model_and_save(fit_id="SPGM_debug",
                                   fit_file_name=fit_file_name,
                                   parallelize=False,
                                   samples_file_name=samples_file_name,
                                   samples_id="SPGM_debug",
                                   theta_init=theta_init)

In [15]:
theta_true = SPS.theta_orig

In [18]:
model_P = TPGM(R=10, theta=theta_true)
model_Q = TPGM(R=10, theta=theta_fit)
sampler = TPGMGibbsSampler(burn_in=200, thinning_nr=150)

/home/mcio/miniconda3/envs/mpgm/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
init_time = time.time()
kl_div_nodes, mse_nodes = EvalMetrics.node_cond_prob_KL_divergence(model_P, model_Q, sampler)
print(time.time()-init_time)

100%|██████████| 14851/14851 [00:13<00:00, 1096.73it/s]

111.4834406375885
